## Capstone Project

### Modeling Notebook - `Gradient Boosting Trees`  `COMBINED DATASET TREE` from 10/24/19

#### Importing Libraries

In [1]:
%matplotlib inline

# general libraries
import re
import string
import sys
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# importing date libraries
import datetime as dt
import dateutil.parser as dparser

# scikit-learn libraries for preprocessing
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# scikit-learn libraries for constructing pipelines
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin

# scikit-learn libraries for clustering and dimensionality reduction
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.mixture import GaussianMixture

# scikit-learn libraries for evaluation
from sklearn import metrics

# scikit-learn libraries for feature selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import RFECV

# scikit-learn libraries for learning
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_validate, cross_val_score
from sklearn.model_selection import validation_curve
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# saving models
import pickle
from sklearn.externals import joblib

# setting pandas display options
pd.set_option("display.max_columns", 999)
pd.set_option("display.max_rows", 10000)
pd.set_option('display.max_colwidth', 100)
pd.set_option('precision', 5)
pd.options.mode.chained_assignment = None

/Users/nate_velarde/anaconda/envs/py3jp/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


#### Directory/File Structure

In [2]:
sys.version

'3.6.8 |Anaconda, Inc.| (default, Dec 29 2018, 19:04:46) \n[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]'

In [3]:
print ('Running pandas version:', pd.__version__)
print ('Running numpy version:', np.__version__)
print ('Running sklearn version:', sklearn.__version__)

Running pandas version: 0.25.1
Running numpy version: 1.14.2
Running sklearn version: 0.21.3


In [4]:
os.getcwd()

'/Users/nate_velarde/Documents/UC_Berkeley/Courses/W210_Capstone/stroke_project/sandbox/notebooks'

In [5]:
os.chdir('../data')

In [6]:
sorted(os.listdir())

['.DS_Store',
 '273_vs_281_null_count_by_feature.csv',
 '273_vs_281_null_count_by_feature.xlsm',
 'Capstone - Complication list - complete.xlsx',
 'Capstone - STS risk factor list.xlsx',
 'Capstone_Fall_Shannon_Sept2019_request.csv',
 'PREOP_dataset_10_24.pkl',
 'PREOP_dataset_TREE_10_24.pkl',
 'X_A_DREF.pkl',
 'X_A_DREF_TREE_SKLEARN.pkl',
 'X_PREOP_10_24.pkl',
 'X_PREOP_TREE_10_24.pkl',
 'X_dev_A_DREF.pkl',
 'X_dev_A_DREF_TREE_SKLEARN.pkl',
 'X_dev_PREOP_10_24.pkl',
 'X_dev_PREOP_TREE_10_24.pkl',
 'X_test_A_DREF.pkl',
 'X_test_A_DREF_TREE_SKLEARN.pkl',
 'X_test_PREOP_10_24.pkl',
 'X_test_PREOP_TREE_10_24.pkl',
 'X_train_A_DREF.pkl',
 'X_train_A_DREF_TREE_SKLEARN.pkl',
 'X_train_PREOP_10_24.pkl',
 'X_train_PREOP_TREE_10_24.pkl',
 'capstone_STS_risk_factor_features.xlsx',
 'capstone_cleaned_data.csv',
 'capstone_data-version-2.xlsx',
 'capstone_data.xlsx',
 'capstone_data_binarized_outcome.pkl',
 'capstone_data_binarized_outcome.xlsx',
 'capstone_data_binarized_outcome_compressed.pkl',


#### Loading Datasets

#### `X_train`, `y_train`
- designation of `_all` denotes complete feature set

In [7]:
X_train_all = pd.read_pickle('X_train_PREOP_TREE_10_24.pkl')
y_train = pd.read_pickle('y_train_PREOP_TREE_10_24.pkl')

In [8]:
X_train_all.shape, y_train.shape

((34192, 73), (34192,))

#### `X_dev`, `y_dev`
- designation of `_all` denotes complete feature set

In [9]:
X_dev_all = pd.read_pickle('X_dev_PREOP_TREE_10_24.pkl')
y_dev = pd.read_pickle('y_dev_PREOP_TREE_10_24.pkl')

In [10]:
X_dev_all.shape, y_dev.shape

((4274, 73), (4274,))

#### `X_test`, `y_test`
- designation of `_all` denotes complete feature set

In [11]:
X_test_all = pd.read_pickle('X_dev_PREOP_TREE_10_24.pkl')
y_test = pd.read_pickle('y_dev_PREOP_TREE_10_24.pkl')

In [12]:
X_test_all.shape, y_dev.shape

((4274, 73), (4274,))

- validating row count for `COMBINED DATASET TREE` from 10/24/19 - `42,740` total observations

In [13]:
42740 - X_train_all.shape[0] - X_dev_all.shape[0] - X_test_all.shape[0]

0

- last look at the data (`X_train_all`) before modeling

In [14]:
X_train_all.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten,cvdstenrt,cvdstenlft,ArrhythDur_when_Combo
0,-2.12457,0.11594,0.32942,0.15700,-2.65268,-0.35221,-2.27155,-1.62927,2.47597,0.98148,3.58585,10,2,2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,1.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,2,1,1,0,0,0,0,0,0
1,1.08243,-0.82771,-0.95497,-0.39563,-0.10617,-0.13691,0.04597,0.25206,-0.29493,0.33103,0.49141,10,0,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0,0.0,0.0,0.0,0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,1.0,2,1,2,1,0,0,0,0,0,0
2,-0.20037,1.53140,1.56497,0.35976,0.59504,-0.25533,-0.34028,0.32442,-0.67978,0.57495,-0.12748,10,1,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,1.0,1,0,0,0,0,0,0,0,0,1
3,0.44103,-0.31888,-0.21754,-0.04732,0.59504,-0.25533,0.04597,-0.76096,-0.29493,0.16842,-0.64322,11,3,3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,1.0,1,0,2,2,0,0,0,0,0,1
4,-0.75014,-1.05900,0.19268,0.56118,-2.17290,-0.36298,0.04597,0.39678,-0.29493,0.57495,-0.65353,6,4,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,1.0,2,0,1,1,0,1,2,0,2,1


In [15]:
X_train_all.tail()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten,cvdstenrt,cvdstenlft,ArrhythDur_when_Combo
34187,-0.29200,0.58776,0.78848,0.26686,0.04145,-0.25533,0.04597,0.83093,-0.29493,0.00581,0.56361,1,3,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.0,1.0,2,0,1,2,0,1,0,0,0,1
34188,-1.11666,0.60626,-0.14917,-0.31935,-0.51214,-0.47063,-2.07842,0.61386,0.57972,-3.16512,1.62604,7,4,3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,1.0,2,2,1,2,0,1,0,0,0,0
34189,0.62428,-1.05900,0.17314,0.54625,-0.32761,-0.07232,-0.34028,-1.05040,-0.45237,1.38800,-0.12748,3,0,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,1.0,1,2,1,1,0,0,1,3,0,0
34190,0.99080,-1.52157,0.75918,1.25109,-0.19844,-0.47063,0.62535,1.04801,-0.67978,0.57495,2.03863,10,3,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.0,0.0,0,0,2,3,0,0,0,0,0,0
34191,-0.20037,0.60626,0.71034,0.21143,-0.69667,-0.04003,-0.14715,3.14642,-0.36491,0.81886,-0.95266,12,1,3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,1.0,2,0,1,0,0,1,0,0,0,1


## `GradientBoostingClassifier`
- see `Muller and Guiido` pages 88-92
- the advice from `Muller and Guido` is "as both gradient boosting and random forests perform well on similar types of data, a common approach is to first try random forests, which work quite robustly.  If random forests work well, but prediction time is at a premium, or it is important to squeeze out the last percentage of accuracy from the machine learning model, moving to gradient boosting often helps." - page 91
- In contrast to random forests, where a higher `n_estimators` is always better, increasing `n_estimators` in gradient boosting leads to a more complex model, which may lead to overfitting.  A common practice is to fit `n_estimators` depending on the time and memory budget, and then search over different `learning_rate`, which controls the degree to which each tree is allowed to correct the mistakes of the previous trees.

- instantiating a `GradientBoostingClassifier()` object

In [17]:
gb_clf = GradientBoostingClassifier(random_state=0,
                                    verbose=2)

- running out of the box with default settings only takes a few seconds

- creating a `StratifiedKFold` cross validation

In [18]:
skf = StratifiedKFold(n_splits=5,
                      shuffle=True,
                      random_state=0)

- defining `scoring_metrics`

In [19]:
scoring_metrics = ['accuracy',
                   'f1',
                   'f1_macro',
                   'f1_weighted',
                   'precision',
                   'precision_macro',
                   'precision_weighted',
                   'recall',
                   'recall_macro',
                   'recall_weighted',
                   'roc_auc']

- conduct `StratifiedKFold` cross-validation

In [20]:
skf_results = cross_validate(gb_clf, # model
                             X_train_all, # feature matrix - X
                             y_train, # target vector - y
                             cv=skf, # cross-validation technique
                             scoring=scoring_metrics, # loss functions
                             return_train_score=True, # returns training score
                             verbose=2, # verbosity level to check progress of calculations
                             n_jobs=-1) # uses all computer cores                     

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   23.5s finished


In [21]:
all_features_results = pd.DataFrame(skf_results).drop(['fit_time', 'score_time'], axis=1)
all_features_results_summary = pd.DataFrame(all_features_results.T)
all_features_results_summary['mean_score'] = all_features_results_summary.mean(axis=1)
all_features_results_summary

,0,1,2,3,4,mean_score
test_accuracy,0.98026,0.98114,0.98216,0.98245,0.98216,0.98163
train_accuracy,0.98461,0.98432,0.98421,0.98428,0.98457,0.98440
test_f1,0.01460,0.00000,0.00000,0.00000,0.00000,0.00292
train_f1,0.17613,0.14371,0.13600,0.14343,0.17255,0.15436
test_f1_macro,0.50231,0.49524,0.49550,0.49557,0.49550,0.49683
train_f1_macro,0.58418,0.56790,0.56402,0.56775,0.58238,0.57324
test_f1_weighted,0.97334,0.97353,0.97419,0.97433,0.97419,0.97392
train_f1_weighted,0.97836,0.97766,0.97745,0.97761,0.97825,0.97787
test_precision,0.05000,0.00000,0.00000,0.00000,0.00000,0.01000
train_precision,0.97826,1.00000,1.00000,1.00000,1.00000,0.99565


#### Key Takeaways
- out of the box `roc_auc` better than `RandomForestClassifier`
- model seems to be overfitting, just like `RandomForestClassifier`
- in `RandomForest` we tuned `max_depth` parameter to see if that reduces overfitting - pre-prune trees
- in `GradientBoosting` the default for `max_depth` is `3`
- in `RandomForest` we found `2` improved results

#### Varying `learning_rate`

- instantiating a `GradientBoostingClassifier()` object

In [22]:
gb_clf = GradientBoostingClassifier(learning_rate=0.3,
                                    random_state=0,
                                    verbose=2)

In [23]:
skf_results = cross_validate(gb_clf, # model
                             X_train_all, # feature matrix - X
                             y_train, # target vector - y
                             cv=skf, # cross-validation technique
                             scoring=scoring_metrics, # loss functions
                             return_train_score=True, # returns training score
                             verbose=2, # verbosity level to check progress of calculations
                             n_jobs=-1) # uses all computer cores                     

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   24.2s finished


In [24]:
all_features_results = pd.DataFrame(skf_results).drop(['fit_time', 'score_time'], axis=1)
all_features_results_summary = pd.DataFrame(all_features_results.T)
all_features_results_summary['mean_score'] = all_features_results_summary.mean(axis=1)
all_features_results_summary

,0,1,2,3,4,mean_score
test_accuracy,0.97938,0.97792,0.97923,0.97821,0.97748,0.97845
train_accuracy,0.98706,0.98640,0.98691,0.98658,0.98662,0.98671
test_f1,0.04082,0.05031,0.00000,0.00000,0.01282,0.02079
train_f1,0.39796,0.33808,0.38062,0.36615,0.37113,0.37079
test_f1_macro,0.51520,0.51957,0.49475,0.49449,0.50071,0.50495
train_f1_macro,0.69571,0.66560,0.68700,0.67968,0.68219,0.68204
test_f1_weighted,0.97335,0.97277,0.97272,0.97221,0.97206,0.97262
train_f1_weighted,0.98334,0.98199,0.98295,0.98254,0.98264,0.98269
test_precision,0.10000,0.09524,0.00000,0.00000,0.02500,0.04405
train_precision,0.95122,0.97938,0.98214,0.93805,0.93103,0.95637


#### Key Takeaways
- increasing `learning_rate` made results worse!

#### Varying `max_depth`

- instantiating a `GradientBoostingClassifier` object

In [31]:
gb_clf = GradientBoostingClassifier(max_depth=2,      # default is `3`
                                    random_state=0,
                                    verbose=2)

In [32]:
skf_results = cross_validate(gb_clf, # model
                             X_train_all, # feature matrix - X
                             y_train, # target vector - y
                             cv=skf, # cross-validation technique
                             scoring=scoring_metrics, # loss functions
                             return_train_score=True, # returns training score
                             verbose=2, # verbosity level to check progress of calculations
                             n_jobs=-1) # uses all computer cores              

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   17.2s finished


In [33]:
all_features_results = pd.DataFrame(skf_results).drop(['fit_time', 'score_time'], axis=1)
all_features_results_summary = pd.DataFrame(all_features_results.T)
all_features_results_summary['mean_score'] = all_features_results_summary.mean(axis=1)
all_features_results_summary

,0,1,2,3,4,mean_score
test_accuracy,0.98231,0.98231,0.98260,0.98304,0.98274,0.98260
train_accuracy,0.98337,0.98340,0.98333,0.98326,0.98337,0.98334
test_f1,0.01626,0.00000,0.00000,0.00000,0.00000,0.00325
train_f1,0.04211,0.04622,0.04202,0.03376,0.04612,0.04204
test_f1_macro,0.50367,0.49554,0.49561,0.49572,0.49565,0.49724
train_f1_macro,0.51686,0.51892,0.51680,0.51266,0.51887,0.51682
test_f1_weighted,0.97440,0.97412,0.97441,0.97463,0.97448,0.97441
train_f1_weighted,0.97547,0.97556,0.97541,0.97524,0.97550,0.97544
test_precision,0.16667,0.00000,0.00000,0.00000,0.00000,0.03333
train_precision,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000


#### Key Takeaways
- lowering `max_depth` to `2` from the default of `3` improved results to the best levels of `RandomForest`

#### Increasing `min_samples_split` to `5` from default of `2`
- in `RandomForest` we found that `2` improved results

- instantiating a `GradientBoostingClassifier` object

In [35]:
gb_clf = GradientBoostingClassifier(max_depth=2,         # default is `3`
                                    min_samples_split=5,  # default is `2`
                                    random_state=0,
                                    verbose=2)

In [36]:
skf_results = cross_validate(gb_clf, # model
                             X_train_all, # feature matrix - X
                             y_train, # target vector - y
                             cv=skf, # cross-validation technique
                             scoring=scoring_metrics, # loss functions
                             return_train_score=True, # returns training score
                             verbose=2, # verbosity level to check progress of calculations
                             n_jobs=-1) # uses all computer cores              

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   16.3s finished


In [37]:
all_features_results = pd.DataFrame(skf_results).drop(['fit_time', 'score_time'], axis=1)
all_features_results_summary = pd.DataFrame(all_features_results.T)
all_features_results_summary['mean_score'] = all_features_results_summary.mean(axis=1)
all_features_results_summary

,0,1,2,3,4,mean_score
test_accuracy,0.98231,0.98231,0.98260,0.98304,0.98274,0.98260
train_accuracy,0.98337,0.98340,0.98333,0.98326,0.98337,0.98334
test_f1,0.01626,0.00000,0.00000,0.00000,0.00000,0.00325
train_f1,0.04211,0.04622,0.04202,0.03376,0.04612,0.04204
test_f1_macro,0.50367,0.49554,0.49561,0.49572,0.49565,0.49724
train_f1_macro,0.51686,0.51892,0.51680,0.51266,0.51887,0.51682
test_f1_weighted,0.97440,0.97412,0.97441,0.97463,0.97448,0.97441
train_f1_weighted,0.97547,0.97556,0.97541,0.97524,0.97550,0.97544
test_precision,0.16667,0.00000,0.00000,0.00000,0.00000,0.03333
train_precision,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000


#### Key Takeaways
- no material change

### Final Takeaways
- `GradientBoostingClassifier` did not meaningfully improve results vis a vis `RandomForestClassifier`, `LogisticRegression` and the `STS` model in terms of `roc_auc` scores